In [1]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [2]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [117]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 20
n_iter = 13000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [148]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.NN as nn

class RNN(nn.NN):

    def __init__(self, D, H, char2idx, idx2char):
        self.D = D
        self.H = H
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        super().__init__(D, D, H, None, None, loss='cross_ent', nonlin='relu')

    def initial_state(self):
        return np.zeros((1, self.H))

    def forward(self, X, h, train=True):
        Wxh, Whh, Why = self.model['Wxh'], self.model['Whh'], self.model['Why']
        bh, by = self.model['bh'], self.model['by']

        X_one_hot = np.zeros(self.D)
        X_one_hot[X] = 1.
        X_one_hot = X_one_hot.reshape(1, -1)

        hprev = h.copy()

        h, h_cache = l.tanh_forward(X_one_hot @ Wxh + hprev @ Whh + bh)
        y, y_cache = l.fc_forward(h, Why, by)

        cache = (X_one_hot, Whh, h, hprev, y, h_cache, y_cache)

        if not train:
            y = util.softmax(y)

        return y, h, cache

    def backward(self, dy, dh_next, cache):
        X, Whh, h, hprev, y, h_cache, y_cache = cache

        # Hidden to output gradient
        dh, dWhy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dby = dby.reshape((1, -1))

        # tanh
        dh = l.tanh_backward(dh, h_cache)

        # Hidden gradient
        dbh = dh
        dWhh = hprev.T @ dh
        dWxh = X.T @ dh
        dh_next = dh @ Whh.T

        grad = dict(Wxh=dWxh, Whh=dWhh, Why=dWhy, bh=dbh, by=dby)

        return grad, dh_next

    def train_step_fwd(self, X_train, y_train, h):

        # Forward 1st layer
        ys, caches = [], []
        for x in X_train:
            y_pred, h, cache = self.forward(x, h, train=True)
            ys.append(y_pred)
            caches.append(cache)

        # loss/error function: last layer
        loss=0.0
        for y_pred, y in zip(ys, y_train):
            loss += loss_fun.cross_entropy(self.model, y_pred, y, lam=0)
        loss /= X_train.shape[0]
        
        return ys, caches, loss

    def train_step_bwd(self, X_train, y_train, ys, caches):

        # last layer dy
        dys=[]
        for y_pred, y in zip(ys, y_train):
            dy = loss_fun.dcross_entropy(y_pred, y)
            dys.append(dy)

        # Backward 1st layer
        dh_next = np.zeros((1, self.H))
        grads = {k: np.zeros_like(v) for k, v in self.model.items()}

        for t in reversed(range(len(ys))):
            grad, dh_next = self.backward(dys[t], dh_next, caches[t])
            for k in grads.keys():
                grads[k] += grad[k]

        # Clipping grads for exploding grad problems
        for k, v in grads.items():
            grads[k] = np.clip(v, -5., 5.)

        return grads

    def sample(self, X_seed, h, size=100):
        chars = [self.idx2char[X_seed]]
        idx_list = list(range(self.vocab_size))
        X = X_seed

        for _ in range(size - 1):
            prob, h, _ = self.forward(X, h, train=False)
            idx = np.random.choice(idx_list, p=prob.ravel())
            chars.append(self.idx2char[idx])
            X = idx

        return ''.join(chars)

    def _init_model(self, D, C, H):
        self.model = dict(
            Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
            Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
            Why=np.random.randn(H, D) / np.sqrt(C / 2.),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
        )

In [149]:
net = RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [150]:
import numpy as np
import impl.utils as util
import impl.constant as c
import copy
from sklearn.utils import shuffle as skshuffle

def get_minibatch(X, y, minibatch_size, shuffle=True):
    minibatches = []

    if shuffle:
        X, y = skshuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha=0.001, mb_size=256, n_iter=2000, print_after=100):
    M = {k: np.zeros_like(v) for k, v in nn.model.items()}
    R = {k: np.zeros_like(v) for k, v in nn.model.items()}
    beta1 = .9
    beta2 = .999

    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)

    idx = 0
    state = nn.initial_state()
    smooth_loss = -np.log(1.0 / len(set(X_train)))

    for iter in range(1, n_iter + 1):
        t = iter

        if idx >= len(minibatches):
            idx = 0
            state = nn.initial_state()

        X_mini, y_mini = minibatches[idx]
        idx += 1

        if iter % print_after == 0:
            print("=========================================================================")
            print('Iter-{} loss: {:.4f}'.format(iter, smooth_loss))
            print("=========================================================================")

            sample = nn.sample(X_mini[0], state, 100)
            print(sample)

            print("=========================================================================")
            print()
            print()

        ys, caches, loss = nn.train_step_fwd(X_mini, y_mini, state)
        smooth_loss = 0.999 * smooth_loss + 0.001 * loss

        grad = nn.train_step_bwd(X_mini, y_mini, ys, caches)

        for k in grad:
            M[k] = util.exp_running_avg(M[k], grad[k], beta1)
            R[k] = util.exp_running_avg(R[k], grad[k]**2, beta2)

            m_k_hat = M[k] / (1. - beta1**(t))
            r_k_hat = R[k] / (1. - beta2**(t))

            nn.model[k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)

    return nn

In [151]:
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, 
                print_after=print_after)

Iter-1000 loss: 3.4469
iets der s tny tou ni iortp nA ertic ihrlsf ifdo8iJ whiltKeNe l un bles bs sax nol neph6uJ gir. Jr t


Iter-2000 loss: 2.7484
ocborlyx, Bhed vncty 'alehe the Wnd 1atolety yed Jfn thers tas an olhgi-litpes0sd fbg the falat inge


Iter-3000 loss: 2.2630
the marlas esmioln. Ahes, irhartret lih"tsy irains, The wres am talatee, lhuperrokt Cn" 1erarinkt en


Iter-4000 loss: 1.9208
xd seileman  Jhe'n tipe efec peril ts pment5: mag the Gutea ane whred tn lpeloland. A9, thre iot 1ug


Iter-5000 loss: 1.6868
es in the couy iytge5 aogard Rarleks and theeand, Nfeitithkeand ihat9sis and un al roed if the apre 


Iter-6000 loss: 1.5067
d cithe pomlte ofent red Ly Jfeap cndexfLJaperlaCyd fshat milela iol GLaka incgiog Gs Aacan 1947, Ja


Iter-7000 loss: 1.3712
eatery my portelsin thes apentory pepuotho onte in opopeex acjtea iry apee iokme iosion"opy, whe "ly


Iter-8000 loss: 1.2686
e Eiro uf iJa an bytweg 1aceng and Iter. fic tidon The uof innte farghve teenary thothol-sthy 9

In [ ]:
=========================================================================
Iter-5000 loss: 1.7817
=========================================================================
eThe courd a本 se ove can Slxto aras chhiind ist erear, she a8"- fite prored candinked vintod coban l
=========================================================================


=========================================================================
Iter-6000 loss: 1.6194
=========================================================================
d pirotor af onveu, anthise ap lorog Jatg atilatins is ly wedt on pus xer, thoscoaugest 141 bad ss A
=========================================================================


=========================================================================
Iter-7000 loss: 1.4840
=========================================================================
es Toky, 2ovevongest ar ssururthesiohe Grokthomicctainas orghobaivenslatouthed.difinfe ih Tnviritarr
=========================================================================


=========================================================================
Iter-8000 loss: 1.3743
=========================================================================
e Efr find hicecentinud an the nf urilotpty Niloeo indadkin, wad Ses an imtires, P and Is ionsed S w
=========================================================================


=========================================================================
Iter-9000 loss: 1.2725
=========================================================================
cendise on lasiond isiately of Jaico rlaigith makae wasth. Nsbyo Nhacandtio tomighiv was WarNl poper
=========================================================================


=========================================================================
Iter-10000 loss: 1.1952
=========================================================================
ed GDeloroergucdar himinan Japan waswus he and in 1247. Japan Na foreltbooped in the nalathi an werl
=========================================================================


=========================================================================
Iter-11000 loss: 1.1147
=========================================================================
ed usghin corctorthe Embertithenthg cely bictroled ind Svesea, Oopan sirunite of Nshigestis Pare Chi
=========================================================================


=========================================================================
Iter-12000 loss: 1.0455
=========================================================================
d Chind Ry come hexpandec, peleedinat sed ar aid was onde w rerir dicon topuctlyt maletd purmkcen ch
=========================================================================


=========================================================================
Iter-13000 loss: 0.9838
=========================================================================
te-Jamanges mert. Ssiceuntr theas Ogeves lakke pstreutian Japan bas meity nu hIn -JaDgest lyUpe iggt
=========================================================================


Out[129]:
<__main__.RNN at 0x7f00f78b9630>

In [5]:
x= []
for i in range(5):
    for j in range(3):
        x.append

In [9]:
class initTest:    
    def __init__(self, D, C, H, depth):
        self.models = []
        for layer in range(depth):
            model = dict(
                Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
                Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
                Why=np.random.randn(H, D) / np.sqrt(C / 2.),
                bh=np.zeros((1, H)),
                by=np.zeros((1, D))
            )
            self.models.append(model)

In [10]:
test = initTest(C=vocab_size, D=vocab_size, depth=2, H=H)
test.models[0].items()
test.models[0]['by']
# for item in test.models[1]:
#     print(item)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.]])